In [24]:
from ortools.linear_solver import pywraplp
from random import randrange,choice
horario = pywraplp.Solver.CreateSolver('SCIP')

S,T,P,C = 5,20,10,100
disp = []
h= [0,1,1,1]
for c in range(C):
    disp.append([])
    for t in range(T):
        disp[c].append(choice(h))
#print(disp)
#A disponibilidade de cada participante, incluindo o lider. Essa disponibilidade   é um conjunto de “slots” 
#representada numa matriz booleana de acessibilidade com uma linha por cada participante 1..C e 
#uma coluna por “slot” 1..T
#disp [C][T] = []

#O conjunto de colaboradores de cada projeto, o seu líder e o número mínimo  de reuniões semanais.
#Nesse caso para um projeto de 0 - P , está associado os colaboradores 
# ((1,2,0),1,3),((4,3,2),4,3),((4,5,6,7,8,9),6,2),((1,8,9,10,11,12,13,14),11,2),((13,15,16,17,18,19),13,2)
proj = [] 
lideres = []
for p in range(P):
    colab = set()
    for col in range(14):
        colab.add(randrange(C))
    cola = list(colab)
    l = random.choice(cola)
    lideres.append(l)
    hora = randrange(1,5)
    proj.append((cola,l,hora))
    

print(proj)




x = {}
for s in range(S):
    x[s]= {}
    for t in range(T):
        x[s][t] = {}
        for p in range(P):
            x[s][t][p] = {}
            for c in range(C):
                x[s][t][p][c] = horario.BoolVar(f'x[{s},{t},{p},{c}]')

def X(s,t,p,c):              # abreviatura
    return x[s][t][p][c] 
#print(round(len(proj[0][0])/2))

#Adiciona condições para que se um colaborador não esteja disponivel para certo slot
#ele não posso ser alocada a esse spot
for c in range(C):
    for t in range(T):
        if disp[c][t] == 0:
            for p in range(P):
                for s in range(S):
                    horario.Add((X(s,t,p,c))== 0 )

#cada colaborador nao pode participar de um projeto que nao é seu
for c in range(C):
    for p in range(P):
        if c not in proj[p][0]:
            for s in range(S):
                for t in range(T):
                    horario.Add((X(s,t,p,c))== 0)   


            

#checa se o mesmo colaborador esta fazendo o trabalho em 2 projetos diferentes ao mesmo tempo
for c in range(C):
    for t in range(T):
        horario.Add(sum(X(s,t,p,c) for p in range(P) for s in range(S))<= 1 )

#Adiciona condições para que se o lider não esteja disponivel não aconteça o projeto   
for p in range(P):
    l = proj[p][1]
    for d in disp[l]:
        if d == 0:
            for s in range(S):
                for c in proj[p][0]:
                    horario.Add((X(s,t,p,c))== 0 )
    
# Cada sala não pode ter mais que uma reunião ao mesmo tempo
for s in range(S):
    for t in range(T):
        horario.Add(sum(X(s,t,p,c) for p in range(P) for c in lideres) <= 1)
#Adiciona condições para que pelo menos 50% dos colaboradores de um dado projeto esteja presente em um slot 
#alocado a este projeto
for p in range(P):
    for t in range(T):
        for s in range(S):
            horario.Add(sum(X(s,t,p,c) for c in proj[p][0])  >= (X(s,t,p,proj[p][1])*round(len(proj[p][0])/2)))
     
    
#Adiciona condições para que o número de reunioes por projeto seja maior que o minimo definido
for p in range(P):
    horario.Add ( proj[p][2] <= sum(X(s,t,p,lideres[p]) for s in range (S) for t in range(T)))

    
for p in range(P):
    for s in range(S):
        for t in range(T):
            horario.Minimize(sum(X(s,t,p,c) for c in range(C)))
    

    
    
    
    
h = sum(X(s,t,p,proj[p][1])for s in range(S) for t in range(T) for p in range(P))


horario.Maximize(h)


status = horario.Solve()
print(status)
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    
    #for p in range(P):
    #    for t in range(T):
    #        for s in range(S):
    #            for c in range(C):
    #                print(X(s,t,p,c).solution_value())
    for s in range(S):
        print("Sala ", s, ":")
        for t in range(T):
            for p in range(P):
                if X(s,t,p,proj[p][1]).solution_value() == 1:
                   
                    a = []
                    for c in proj[p][0] :
                        if X(s,t,p,c).solution_value() == True:
                            a.append(c)
                    print("Slot ", t ," Projeto:", p," Colaboradores:", a )


[([32, 2, 99, 4, 68, 35, 74, 14, 18, 52, 87, 88, 58, 29], 87, 3), ([65, 98, 66, 1, 72, 41, 74, 43, 44, 76, 79, 83, 59, 61], 59, 2), ([32, 97, 34, 35, 68, 40, 10, 11, 76, 46, 78, 80, 90], 76, 2), ([32, 67, 36, 5, 35, 8, 44, 77, 12, 54, 87, 56, 29], 54, 4), ([5, 6, 38, 74, 11, 14, 79, 81, 51, 19, 88, 27, 92, 61], 79, 3), ([96, 97, 99, 38, 8, 76, 13, 78, 77, 55, 24, 90, 63], 77, 3), ([96, 97, 32, 6, 39, 76, 15, 56, 62, 30, 57], 57, 2), ([33, 3, 41, 45, 48, 80, 50, 18, 53, 56, 57, 90, 28], 80, 4), ([0, 66, 67, 6, 43, 17, 49, 87, 55, 22, 56, 28, 29, 94], 94, 1), ([98, 67, 39, 91, 75, 11, 49, 83, 20, 85, 56, 26, 59], 20, 4)]
0
Sala  0 :
Slot  0  Projeto: 8  Colaboradores: [0, 66, 43, 49, 22, 28, 94]
Slot  1  Projeto: 3  Colaboradores: [36, 35, 8, 12, 54, 56]
Slot  2  Projeto: 6  Colaboradores: [96, 6, 15, 56, 62, 57]
Slot  3  Projeto: 1  Colaboradores: [65, 98, 1, 74, 44, 59, 61]
Slot  4  Projeto: 2  Colaboradores: [34, 68, 10, 11, 76, 90]
Slot  5  Projeto: 5  Colaboradores: [8, 77, 55, 24, 